# VacationPy
----

#### Analysis
Based off our data and maps I would personally enjoy some amazing tempratures with no clouds at Sunset Point Resort. They are located Calle Vista Del Sol, Entrada Don Goyo Residencia and they carry a rating of 4.7 out of 5. Their exact coordinates are (15.8693331, -97.0726428). See you there!

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
df=pd.read_csv("output_data/cities.csv")
df.head()


,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness
0,vanderhoof,54.017529,-124.007663,Clouds,13.91,89,1024,3.18,90
1,saskylakh,71.964953,114.093943,Clouds,-28.71,100,1030,3.74,100
2,burnie,-41.052369,145.906407,Clouds,59.14,69,1001,18.66,96
3,chokurdakh,70.619382,147.902397,Clouds,-41.13,100,1034,1.99,95
4,dingle,11.000076,122.671784,Clouds,83.17,79,1010,10.71,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# get data
data = df.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)



# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
clear_mask=(df.wind_speed<10) & (df.temperature<80) & (df.temperature>70)& (df.cloudiness==0)
resorts_df=df.loc[clear_mask].reset_index(drop=True)
resorts_df

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness
0,busselton,-33.644499,115.348875,Clear,76.93,35,1016,5.61,0
1,kapaa,22.074733,-159.319160,Clear,70.29,82,1015,3.44,0
2,makakilo city,21.346944,-158.085833,Clear,74.23,68,1016,8.05,0
3,kahului,20.889410,-156.472712,Clear,75.42,80,1016,3.44,0
4,cayenne,4.937114,-52.325831,Clear,73.42,94,1010,2.30,0
5,puerto suarez,-18.963179,-57.797455,Clear,74.88,46,1006,2.42,0
6,cabo san lucas,22.893888,-109.920060,Clear,71.04,66,1009,1.99,0
7,puerto escondido,15.869333,-97.072643,Clear,76.71,73,1014,5.75,0
8,pokaran,26.919881,71.919757,Clear,79.27,12,1013,3.74,0
9,iranshahr,27.206990,60.688251,Clear,71.89,18,1017,4.61,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
resorts = []

for indx, row in hotels_df.iterrows():
    lat = row.latitude
    lng = row.longitude

    #geocoord
    target_coordinates = f"{lat}, {lng}"
    target_search = "resort"
    target_radius = 5000
    target_type = "lodging"

    #parameters
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    #url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #request using params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"]) > 0):
        resort = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
    else:
        resort = None
        address = None
        rating = np.nan
        user_ratings_total = np.nan
        
    resort = {"resort" : resort,
            "address": address,
            "rating": rating,
            "user_ratings_total": user_ratings_total}
    
    resorts.append(resort)

In [23]:
r_df=pd.DataFrame(resorts)
r_df.head()

,resort,address,rating,user_ratings_total
0,The Royal Palms Resort,"8 Albert St, Busselton",4.4,133.0
1,Kaha Lani Resort,"4460 Nehe Rd, Lihue",4.5,317.0
2,Beach Villas at Ko Olina AOAO,"92-106 Waialii Pl, Kapolei",4.8,42.0
3,Paradise Resorts,"40 Kunihi Ln #227, Kahului",5.0,1.0
4,None,None,NaN,NaN


In [25]:
final_resort=pd.merge(resorts_df,r_df,left_index=True, right_index=True)
final_resort

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,cloudiness,resort,address,rating,user_ratings_total
0,busselton,-33.644499,115.348875,Clear,76.93,35,1016,5.61,0,The Royal Palms Resort,"8 Albert St, Busselton",4.4,133.0
1,kapaa,22.074733,-159.319160,Clear,70.29,82,1015,3.44,0,Kaha Lani Resort,"4460 Nehe Rd, Lihue",4.5,317.0
2,makakilo city,21.346944,-158.085833,Clear,74.23,68,1016,8.05,0,Beach Villas at Ko Olina AOAO,"92-106 Waialii Pl, Kapolei",4.8,42.0
3,kahului,20.889410,-156.472712,Clear,75.42,80,1016,3.44,0,Paradise Resorts,"40 Kunihi Ln #227, Kahului",5.0,1.0
4,cayenne,4.937114,-52.325831,Clear,73.42,94,1010,2.30,0,None,None,NaN,NaN
5,puerto suarez,-18.963179,-57.797455,Clear,74.88,46,1006,2.42,0,None,None,NaN,NaN
6,cabo san lucas,22.893888,-109.920060,Clear,71.04,66,1009,1.99,0,Solmar Resort,"Av Solmar #1, Centro, Cabo San Lucas",4.5,3175.0
7,puerto escondido,15.869333,-97.072643,Clear,76.71,73,1014,5.75,0,Sunset Point Resort,"Calle Vista Del Sol, Entrada Don Goyo Residencia",4.7,46.0
8,pokaran,26.919881,71.919757,Clear,79.27,12,1013,3.74,0,Pokaran Desert Resort,"Jaisalmer - Jodhpur Road, Dist, Pokhran",3.7,362.0
9,iranshahr,27.206990,60.688251,Clear,71.89,18,1017,4.61,0,None,None,NaN,NaN


In [28]:
infos = []

for indx, row in final_resort.iterrows():
    info_box_template = f"""<dl>
                        <dt>Resort Name:</dt><dd>{row.resort}</dd>
                        <dt>Rating:</dt><dd>{row.rating}</dd>
                        <dt>Address:</dt><dd>{row.address}</dd>
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                        </dl>
                        """
    infos.append(info_box_template)

In [29]:
 # get data
data = df.loc[:, ["latitude", "longitude"]]
resort_data = final_resort.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(resort_data, info_box_content=infos)
fig.add_layer(markers)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…